In [34]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings('ignore')

alt.data_transformers.disable_max_rows()
genetic = pd.read_csv('test_genetic_disorders.csv')

# defining columns
blood_cell_col = 'Blood cell count (mcL)'
white_cell_col = 'White Blood cell count (thousand per microliter)'
gender_col = 'Gender'
inherit_mother_col = "Genes in mother's side"
inherit_father_col = 'Inherited from father'

disorder_col = 'Blood test result'  

all_cols = [blood_cell_col, white_cell_col, gender_col, 
            inherit_mother_col, inherit_father_col, disorder_col]

# cleaning data
genetic_clean = genetic[all_cols].copy()

for col in all_cols:
    genetic_clean[col] = genetic_clean[col].replace(-99, pd.NA)
    
for col in [gender_col, inherit_mother_col, inherit_father_col, disorder_col]:
    genetic_clean[col] = genetic_clean[col].replace('-99', pd.NA)
    
genetic_clean = genetic_clean.dropna()

valid_results = ['abnormal', 'normal', 'slightly abnormal', 'inconclusive']
genetic_clean = genetic_clean[genetic_clean[disorder_col].isin(valid_results)]

# creating an inheritance source
def get_inheritance_source(row):
    mother = row[inherit_mother_col]
    father = row[inherit_father_col]
    
    if mother == 'Yes' and father == 'Yes':
        return 'Both'
    elif mother == 'Yes':
        return 'Maternal'
    elif father == 'Yes':
        return 'Paternal'
    else:
        return 'Neither'

genetic_clean['Inheritance_Source'] = genetic_clean.apply(get_inheritance_source, axis=1)

In [35]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

X = genetic_clean[[blood_cell_col, white_cell_col]].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_clusters = 4  
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
genetic_clean['Cluster'] = kmeans.fit_predict(X_scaled)
genetic_clean['Cluster_Label'] = genetic_clean['Cluster'].apply(
    lambda x: f'Cluster {x+1}'
)

# getting cluster centers 
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)
cluster_centers = pd.DataFrame(
    centers_original,
    columns=[blood_cell_col, white_cell_col]
)
cluster_centers['Cluster_Label'] = [f'Cluster {i+1}' for i in range(n_clusters)]

# scatter with disorder colors
scatter_static = alt.Chart(genetic_clean).mark_circle(
    size=100,
    opacity=0.7
).encode(
    x=alt.X(blood_cell_col + ':Q',
            title='Blood Cell Count (mcL)',
            scale=alt.Scale(zero=False)),
    y=alt.Y(white_cell_col + ':Q',
            title='White Blood Cell Count (thousand per microliter)',
            scale=alt.Scale(zero=False)),
    color=alt.Color(disorder_col + ':N',
                    title='Disorder Type',
                    scale=alt.Scale(scheme='category10')),
    tooltip=[
        alt.Tooltip(blood_cell_col + ':Q', title='Blood Cell Count', format='.3f'),
        alt.Tooltip(white_cell_col + ':Q', title='White Blood Cell Count', format='.3f'),
        alt.Tooltip(disorder_col + ':N', title='Disorder Type'),
        alt.Tooltip('Cluster_Label:N', title='Cluster'),
        alt.Tooltip(gender_col + ':N', title='Gender')
    ]
).properties(
    width=700,
    height=500,
    title={
        "text": "Blood Cell Count vs White Blood Cell Count by Disorder Type",
        "subtitle": "Task 1: Identifying biomarker patterns and diagnostic clusters"
    }
)

# adding cluster centers as large O marks
cluster_centers_chart = alt.Chart(cluster_centers).mark_point(
    size=500,
    shape='circle',
    filled=True,
    color='black',
    strokeWidth=3
).encode(
    x=blood_cell_col + ':Q',
    y=white_cell_col + ':Q',
    tooltip=['Cluster_Label']
)

# combining base + centers
scatter_static_final = (scatter_static + cluster_centers_chart).configure_title(
    fontSize=16,
    anchor='start'
).interactive()

brush = alt.selection_interval()

# main scatter plot with clusters
scatter_points = alt.Chart(genetic_clean).mark_circle(size=100).encode(
    x=alt.X(blood_cell_col + ':Q',
            title='Blood Cell Count (mcL)',
            scale=alt.Scale(zero=False)),
    y=alt.Y(white_cell_col + ':Q',
            title='White Blood Cell Count (thousand per microliter)',
            scale=alt.Scale(zero=False)),
    color=alt.condition(
        brush,
        alt.Color(disorder_col + ':N',
                  title='Disorder Type',
                  scale=alt.Scale(scheme='category10')),
        alt.value('lightgray')
    ),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.2)),
    tooltip=[blood_cell_col, white_cell_col, disorder_col, 'Cluster_Label', gender_col]
).add_params(brush).properties(
    width=700,
    height=400,
    title='Drag to Select Points and See Distribution Below'
)

# cluster centers
centers_on_interactive = alt.Chart(cluster_centers).mark_point(
    size=500,
    shape='circle',
    color='black',
    strokeWidth=3
).encode(
    x=blood_cell_col + ':Q',
    y=white_cell_col + ':Q',
    tooltip=['Cluster_Label']
)

# linked bar chart
scatter_bars = alt.Chart(genetic_clean).mark_bar().encode(
    y=alt.Y(disorder_col + ':N',
            title='Disorder Type',
            sort='-x'),
    x=alt.X('count():Q',
            title='Number of Patients'),
    color=alt.Color(disorder_col + ':N',
                    scale=alt.Scale(scheme='category10'),
                    legend=None),
    tooltip=[disorder_col, 'count()']
).transform_filter(brush).properties(
    width=700,
    height=200,
    title='Count of Selected Patients by Disorder Type'
)

# combining all layers
scatter_interactive = alt.vconcat(
    scatter_points + centers_on_interactive,
    scatter_bars
).resolve_scale(color='independent')

scatter_interactive

alt.VConcatChart(...)

In [36]:
# Static 
# Chart 1: Gender × Disorder Type
bar_gender = alt.Chart(genetic_clean).mark_bar().encode(
    x=alt.X(disorder_col + ':N',
            title='Disorder Type',
            axis=alt.Axis(labelAngle=-45)),
    xOffset=gender_col + ':N',  # This makes bars side-by-side!
    y=alt.Y('count():Q',
            title='Number of Patients'),
    color=alt.Color(gender_col + ':N',
                    title='Gender',
                    scale=alt.Scale(scheme='set1')),
    tooltip=[gender_col, disorder_col, 'count()']
).properties(
    width=600,
    height=350,
    title='Disorder Distribution by Gender (Side-by-Side)'
)

# Chart 2: Inheritance × Disorder Type 
bar_inheritance = alt.Chart(genetic_clean).mark_bar().encode(
    x=alt.X(disorder_col + ':N',
            title='Disorder Type',
            axis=alt.Axis(labelAngle=-45)),
    xOffset='Inheritance_Source:N',  # This makes bars side-by-side!
    y=alt.Y('count():Q',
            title='Number of Patients'),
    color=alt.Color('Inheritance_Source:N',
                    title='Inheritance Pattern',
                    scale=alt.Scale(scheme='tableau10')),
    tooltip=['Inheritance_Source', disorder_col, 'count()']
).properties(
    width=600,
    height=350,
    title='Disorder Distribution by Inheritance Pattern (Side-by-Side)'
)

# combining both static charts
bar_static = alt.vconcat(
    bar_gender,
    bar_inheritance
).properties(
    title='Gender and Inheritance Pattern Analysis'
)

# Interactive chart
# clicking selection parameter
click = alt.selection_point(fields=[gender_col], on='click', empty=True)

# Left chart: Gender × Disorder Type
bar_left = alt.Chart(genetic_clean).mark_bar().encode(
    x=alt.X(gender_col + ':N',
            title='Gender'),
    y=alt.Y('count():Q',
            title='Number of Patients'),
    color=alt.Color(disorder_col + ':N',
                    title='Disorder Type',
                    scale=alt.Scale(scheme='category10')),
    opacity=alt.condition(click, alt.value(1.0), alt.value(0.3)),
    tooltip=[gender_col, disorder_col, 'count()']
).properties(
    width=320,
    height=400,
    title='Click on Bars to Filter'
).add_params(click)

# Right chart: Inheritance × Disorder Type 
bar_right = alt.Chart(genetic_clean).mark_bar().encode(
    x=alt.X('Inheritance_Source:N',
            title='Inheritance Pattern',
            axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count():Q',
            title='Number of Patients'),
    color=alt.Color(disorder_col + ':N',
                    title='Disorder Type',
                    scale=alt.Scale(scheme='category10')),
    opacity=alt.condition(click, alt.value(1.0), alt.value(0.3)),
    tooltip=['Inheritance_Source', disorder_col, 'count()']
).transform_filter(click).properties(
    width=400,
    height=400,
    title='Inheritance Patterns (Filtered by Gender)'
)

# combining
bar_interactive = alt.hconcat(
    bar_left,
    bar_right
).resolve_scale(
    color='shared'
).properties(
    title='Interactive: Gender and Inheritance Pattern Comparison'
)

bar_interactive

alt.HConcatChart(...)